<a href="https://colab.research.google.com/github/AndreyBuyanov/HPC.Lab1/blob/main/HPC_Lab1_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Отчёт https://disk.yandex.ru/i/Ue_SfxYs6mr_NQ

Подключаем GoogleDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Устанавливаем OpenJDK

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

Устанавливаем переменные окружения, необходимые для скачивания дистрибутива Spark

In [ ]:
import os
os.environ['APACHE_SPARK_DISTR_NAME'] = 'spark-3.0.2-bin-hadoop2.7'
os.environ['APACHE_SPARK_URL'] = 'https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz'

Скачиваем и распаковываем архив с дистрибутивом Spark

In [ ]:
![ ! -d $APACHE_SPARK_DISTR_NAME ] \
   && wget -q $APACHE_SPARK_URL \
   && tar xf $APACHE_SPARK_DISTR_NAME.tgz \
   && rm -f $APACHE_SPARK_DISTR_NAME.tgz

Устанавливаем переменные окружения, необходимые для работы дистрибутива Spark

In [ ]:
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.0.2-bin-hadoop2.7'

Устанавливаем пакет findspark

In [ ]:
!pip install findspark > /dev/null

Подключаем пакет findspark

In [ ]:
import findspark
findspark.init()

Подключаем пакет SparkSession

In [ ]:
from pyspark.sql import SparkSession

Создаём новую сессию

In [ ]:
spark = SparkSession.builder.master('local').appName("HPC.Lab1").getOrCreate()

Загружаем данные из исходного датасета в дата-фрейм main_df и показываем

In [ ]:
main_df = spark.read.csv(r"./drive/My Drive/TPU/HPC/Lab1/brooklyn_sales_map.csv", header=True)
main_df.show()

+---+--------+--------------------+-----------------------+---------+-----+----+--------+--------------+--------------------+----------------+--------+-----------------+----------------+-----------+---------+----------+----------+-----------------+----------------------+----------+----------+------------+---------+---+------+------+----------+-------+-------+--------+----------+----------+----------+---------+----------+--------+-------------------+---------+---------+---------+---------+--------+--------+-------+-------+-------+---------+---------+---------+-------+---------+---------+--------------------+-------+--------+-------+-------+----------+----------+----------+---------+----------+---------+----------+--------+---------+--------+----------+--------+--------+---------+---------+---+--------+----------+-------+--------+----------+---------+----------+---------+---------+----------+----------+--------------------+--------+--------+--------+-------+--------+--------+----------+-

Импортируем функции

In [ ]:
from pyspark.sql import functions as F

Задание 1: "Найдите среднюю площадь жилья (gross_sqft) и выведите новую таблицу, содержащую стоимость жилья и отклонение стоимости от среднего значения"

Вычислить среднюю площадь жилья можно с помощью функции avg, перед вызовом нужно выбрать столбец gross_sqft и преобразовать дата-фрейм к типу GrouppedData с помощью метода groupBy(). Затем вызвать метод avg, передав в него название столбца, по которому нужно посчитать среднее значение. Затем, с помощью метода collect() дата-фрейм преобразуется в список объектов Row, представляющих строку дата-фрейма. Первым элементом списка окажется словарь и по ключу 'avg' можно получить искомое среднее значение площади жилья. Выведем полученное значение.

In [ ]:
avg_gross_sqft = main_df                    \
    .select('gross_sqft')                   \
    .groupBy()                              \
    .agg(F.avg('gross_sqft').alias('avg'))  \
    .collect()
avg_gross_sqft_value = avg_gross_sqft[0]['avg']
print(avg_gross_sqft_value)

2970.8766408362603


Вычисление средней стоимости жилья аналогично вычислению средней площади жилья.

Вычисляем среднее значение стоимости жилья.

In [ ]:
avg_sale_price = main_df                    \
    .select('sale_price')                   \
    .groupBy()                              \
    .agg(F.avg('sale_price').alias('avg'))  \
    .collect()
avg_sale_price_value = avg_sale_price[0]['avg']
print(avg_sale_price_value)

506754.47769281344


Создадим новый дата-фрейм, в котором отобразим стоимость жилья и отклонение стоимости от среднего значения. Для этого в исходном дата-фрейме выберем столбец sale_price, содержащий стоимость жилья и добавим столбец, значениями которого будет разность стоимости и среднего значения стоимости. Отобразим получившийся дата-фрейм.

In [ ]:
sale_price_deviation_df = main_df       \
    .select('sale_price')               \
    .withColumn('sale_price_deviation', \
                main_df.sale_price - avg_sale_price_value)
sale_price_deviation_df.show()

+----------+--------------------+
|sale_price|sale_price_deviation|
+----------+--------------------+
| 499401179|4.9889442452230716E8|
|  3.45e+08|3.4449324552230716E8|
|   3.4e+08|3.3949324552230716E8|
| 276947000|2.7644024552230716E8|
| 202500000| 2.019932455223072E8|
| 185445000| 1.849382455223072E8|
|  1.71e+08| 1.704932455223072E8|
|  1.69e+08| 1.684932455223072E8|
|  1.65e+08| 1.644932455223072E8|
|  1.61e+08| 1.604932455223072E8|
|   1.6e+08| 1.594932455223072E8|
|  1.58e+08| 1.574932455223072E8|
| 154250000| 1.537432455223072E8|
| 138106368| 1.375996135223072E8|
| 135212000| 1.347052455223072E8|
|  1.35e+08| 1.344932455223072E8|
| 132400000|1.3189324552230719E8|
|  1.25e+08|1.2449324552230719E8|
|  1.24e+08|1.2349324552230719E8|
| 121313223|1.2080646852230719E8|
+----------+--------------------+
only showing top 20 rows



Задание 2: "Найдите среднюю площадь жилья (gross_sqft) для каждого года, в котором оно было построено (year_built)"

Выберем столбцы 'gross_sqft' и 'year_built', сгруппируем получивщийся дата-фрейм по столбцу 'year_built' и применим функцию avg. В итоге получим дата-фрейм, содержащий год постройки и среднюю площадь жилья за этот год. Отобразим получивщийся дата-фрейм.

In [ ]:
gross_sqft_d_and_year_built_df = main_df                \
    .select(['gross_sqft', 'year_built'])               \
    .groupBy('year_built')                              \
    .agg(F.avg('gross_sqft').alias('avg_gross_sqft'))   \
    .orderBy(F.desc('year_built'))
gross_sqft_d_and_year_built_df.show()

+----------+------------------+
|year_built|    avg_gross_sqft|
+----------+------------------+
|      2017| 3290.056603773585|
|      2016|1495.8328220858896|
|      2015|3419.7989172467132|
|      2014|3222.1512605042017|
|      2013| 3545.206660973527|
|      2012|          2775.272|
|      2011|4290.2784431137725|
|      2010| 4350.191562143672|
|      2009| 4251.070020005716|
|      2008|3004.8798000454444|
|      2007| 2283.872844058523|
|      2006|2155.3391670717874|
|      2005| 2645.361133090679|
|      2004|2432.5135473113796|
|      2003| 1531.779675994109|
|      2002|2252.2347520842477|
|      2001| 3749.083408071749|
|      2000| 3741.164341085271|
|      1999| 3898.143790849673|
|      1998| 2928.172344689379|
+----------+------------------+
only showing top 20 rows



Задание 3: "Найдите среднюю стоимость жилья (sale_price) для всех сочетаний соседств (neighborhood) и категорий класса здания (building_class_category)"

Выберем необходимые столбцы: 'sale_price', 'neighborhood', 'building_class_category'. Сгруппируем получивщийся дата-фрейм по столбцам 'neighborhood' и 'building_class_category' для получения необходимых сочетаний в виде групп и применим к ним функцию avg. В итоге получим дата-фрейм согласно заданию. Отобразим получивщийся дата-фрейм.

In [ ]:
sale_price_group_df = main_df                                           \
    .select(['sale_price', 'neighborhood', 'building_class_category'])  \
    .groupBy(['neighborhood', 'building_class_category'])               \
    .agg(F.avg('sale_price').alias('avg_sale_price'))                   \
    .orderBy(F.desc('avg_sale_price'))
sale_price_group_df.show()

+--------------------+-----------------------+-------------------+
|        neighborhood|building_class_category|     avg_sale_price|
+--------------------+-----------------------+-------------------+
|    BROOKLYN HEIGHTS|     23  LOFT BUILDINGS|           2.0502E8|
|    BROOKLYN HEIGHTS|        26 OTHER HOTELS|             1.45E8|
|          MILL BASIN|    22  STORE BUILDINGS|         1.384735E8|
|  DOWNTOWN-METROTECH|        26 OTHER HOTELS|             1.35E8|
|    BROOKLYN HEIGHTS|    21 OFFICE BUILDINGS|             9.45E7|
|    BROOKLYN-UNKNOWN|   21  OFFICE BUILDINGS|              9.0E7|
|DOWNTOWN-FULTON MALL|       45  CONDO HOTELS|              8.8E7|
|DOWNTOWN-FULTON F...|   35  INDOOR PUBLIC...|              6.5E7|
|  WILLIAMSBURG-SOUTH|   46  CONDO STORE B...|              5.9E7|
|            CANARSIE|          30 WAREHOUSES|              5.3E7|
|       BUSH TERMINAL|   39  TRANSPORTATIO...|        4.8327118E7|
|    BROOKLYN HEIGHTS|       26  OTHER HOTELS|         4.72062

Задание 4: "В исходном датафрейме удалите все строчки с записями домов, которые были построены позже 2000 года (year_built), а также те, которые содержат только нулевые значения"

К исходному дата-фрейму добавим столбец, содержащий год постройки жилья виде целых чисел. Это необходимо для сравнения, поскольку год постройки в исходном дата-фрейме представлен в виде строки. Затем, сделаем выборку с условием "год постройки" меньше 2000 или не равен 0. Выведем получившийся дата-фрейм.

In [ ]:
year_built_lt_2000_and_not_null_df = main_df                                \
    .withColumn('year_built_i', main_df['year_built'].cast('int'))          \
    .where(F.col('year_built_i') < 2000)                                    \
    .where(F.col('year_built_i') != 0)                                      \
    .drop('year_built_i')
year_built_lt_2000_and_not_null_df.show()

+---+--------+--------------------+-----------------------+---------+-----+----+--------+--------------+--------------------+----------------+--------+-----------------+----------------+-----------+---------+----------+----------+-----------------+----------------------+----------+----------+------------+---------+---+------+------+----------+-------+-------+--------+----------+----------+----------+---------+----------+--------+--------------------+---------+---------+---------+---------+--------+--------+-------+-------+-------+---------+---------+---------+-------+---------+---------+--------------------+-------+--------+-------+-------+----------+----------+----------+---------+----------+---------+----------+--------+---------+--------+----------+--------+--------+---------+---------+---+--------+----------+-------+--------+----------+---------+----------+---------+---------+----------+----------+--------------------+--------+--------+--------+-------+--------+--------+----------+